Import das bibliotecas necessárias para realizar os procedimentos

In [49]:
import MySQLdb
import sqlalchemy
import pandas as pd 
import os

Configurando a conexão com o banco de dados

In [50]:
#SQL Connection 
user = 'jonathas' #usuário
password = '123456' #senha
host = 'localhost' #manter como está
database = 'projeto2IGTI' #schema
string_conexao = f'mysql://{user}:{password}@{host}/{database}' #url de conexão com o bacanco de dados, não mudar.

Realizando a conexão com o banco de dados 

In [51]:
engine = sqlalchemy.create_engine(string_conexao)
conn = engine.connect()



Nesse momento vamos ler os arquivos que estão dentro da pasta chamada base.

In [52]:
#Criar Lista de arquivos
lista_df = [] # cria uma lista
lista_path = os.listdir('./base') #mostra qual a pasta
lista_path[:5] #mostra os 5 primeiro arquivos que estão dentro da pasta base

#ler os arquivos a armazena num dataframe
for arquivo in lista_path: # Esse for vai percorrer todos os arquivos dentro da pasta base
    local_arquivo = os.path.join('./base', arquivo)
    df = pd.read_csv(local_arquivo, sep = '|', encoding='latin-1') #o conteúdo dos arquivos está separado por um  |, para ter certerza, abra um dos arquivos e observe qual o separador.
    lista_df.append(df) 
df_pesquisa = pd.concat(lista_df) # Atribui os dados coletados em um dataframe

df_pesquisa[:5] #mostra as 5 primeiras linhas do dataframe gerado


,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies
0,10141743,2020-01-01,Masculino,1981-04-25,peixe,frio,Cerveja,Praticar esporte
1,10141860,2020-01-01,Feminino,1966-01-19,peixe,quente,Café,Assistir TV
2,10142548,2020-01-01,Masculino,1973-08-02,tartaruga,frio,Café,Ler livros
3,10142591,2020-01-01,Masculino,1997-03-05,gato,moderado,Cerveja,Aprender algo novo
4,10142654,2020-01-01,Feminino,1969-08-05,tartaruga,moderado,Café,Assistir TV




Inicialmente vamos trabalhar os dados que não possuem dependecias de outras tabelas.
Essa tal independência pode ser observada no schema do enunciado do trabalho.

In [53]:
#Inserir dados no banco
#Para pessoas, podemos observar que a tabela de pessoas possui as seguintes características cod_pessoa, genero e data_nascimento.
#O que fazemos aqui é criar um dataframe de pessoas com essses dados, onde selecionamos as colunas correspondentes no df_pesquisa
df_pessoas = df_pesquisa[['cod_pessoa', 'genero', 'data_nascimento']].drop_duplicates() #o drop_duplicates remove qualquer conteúdo que esteja duplicado

#Podemos observar que o animal só tem a espécie dele para ser armazenada, que no caso está referida como "animal_estimacao"
#Como é apenas uma coluna que será armazenada, iremos usar uma lista, se fossem mais, utilizaríamos um dataframe.
animais = list(df_pesquisa.animal_estimacao.unique()) #Como o nome dos animais se repete ao longo das linhas da pesquisa, utilizamos a fução unique() para selecionar apenas os valores unicos, sem repetições.

#A mesma lógica utilizada com os animais pode ser utilizada com hobbies, bebidas e climas.
hobbies = list(df_pesquisa.hobbies.unique())
bebidas = list(df_pesquisa.bebida_favorita.unique())
climas = list(df_pesquisa.clima.unique())



Vamos verificar o dataframe de pessoas e as listas de animais, hobbies, bebidas e climas.
Para isso remova a tag comentário '#' do comando para visualizar cada uma das variáveis.

In [54]:
df_pessoas[:5] #mostra as 5 primeiras linhas
#animais
#hobbies
#bebidas
#climas


,cod_pessoa,genero,data_nascimento
0,10141743,Masculino,1981-04-25
1,10141860,Feminino,1966-01-19
2,10142548,Masculino,1973-08-02
3,10142591,Masculino,1997-03-05
4,10142654,Feminino,1969-08-05


A partir desse ponto, se supõe que o banco de dados está conectado perfeitamente e que as tabelas já estão criadas.
Caso não esteja, verifique o arquivo scriptPesquisa.sql para criar as tabelas e seu banco de acordo com o enunciado da questão.



Como df_pessoas é um dataframe, utilizaremos a seguinte lógica para inserir as pessoas no banco de dados:

In [55]:
#Inserir pessoas
for pessoa in df_pessoas.itertuples(): #Um for que vai criar uma variável temporária do tipo pessoa que vai percorrer todo o dataframe df_pessoas
    #observando o dataframe df_pessoas podemos notar que, de acordo com as colunas, cada pessoa possui essas três características abaixo
    cod_pessoa = pessoa.cod_pessoa	 #pessoa.cod_pessoa vai referenciar inicialmente o cod_pessoa da pessoa[1] no dataframe de pessoas, e assim por diante.
    genero = pessoa.genero #o mesmo pensamento acima serve para as pessoa.genero e pessoa.data_nascimento
    nascimento = pessoa.data_nascimento
    #A partir daqui, guardamos três valores, cod_pessoa, genero e nascimento.
    try:
        #Nessa primeira etapa, vamos inserir na tabela "pessoa" nas colunas cod_pessoa, genero e data_nascimento as variaveis que guardamos anteriormente, cod_pessoa, genero e nascimento
        querey = f"""insert into pessoa (cod_pessoa, genero, data_nascimento )
                     values ('{cod_pessoa}',  '{genero}', '{nascimento}')"""      
        conn.execute(querey) #enviar soliciatação par inserir      
        print(f"Registro inserido com sucesso: {cod_pessoa}") #Se ocorrer tudo bem
    except Exception as e:        
        print(f"Não foi possível inserir o regristro {cod_pessoa}. O erro foi encontrado foi: {e}") #Se houver algum erro
print("Pessoas Inseridas")

pessoa (cod_pessoa, genero, data_nascimento )
                     values ('10142393',  'Masculino', '1970-09-18')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro 10142971. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry '10142971' for key 'pessoa.PRIMARY'")
[SQL: insert into pessoa (cod_pessoa, genero, data_nascimento )
                     values ('10142971',  'Masculino', '1951-12-26')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro 10143012. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry '10143012' for key 'pessoa.PRIMARY'")
[SQL: insert into pessoa (cod_pessoa, genero, data_nascimento )
                     values ('10143012',  'Masculino', '1968-05-17')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro 10143624. O erro foi encontrado foi: (MySQLdb._except

Agora vamos inserir os dados que estão nas listas: animais, hobbies, climas e bebidas

In [56]:
#Inserir animais
#A lógica é bem parecida com a de pessoas, mas como é apenas uma lista, só iremos inserir um valor
for a in animais: #Nesse caso, como existe apenas um valor, a variável temporária 'a' que está sendo criada no for irá assumir o valor contido em casa linha da lista animais
    try:
        #Nessa primeira etapa, vamos inserir na tabela "animal_estimacao" na coluna animal, o valor de a.
        querey = f"""insert into animal_estimacao (animal) 
                     values ('{a}')"""
        conn.execute(querey)#enviar soliciatação par inserir      
    except Exception as e:
        print(f"Não foi possível inserir o regristro {a}. O erro foi encontrado foi: {e}")  
print("Animais Inseridos")

Não foi possível inserir o regristro peixe. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'peixe' for key 'animal_estimacao.animal_UNIQUE'")
[SQL: insert into animal_estimacao (animal) 
                     values ('peixe')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro tartaruga. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'tartaruga' for key 'animal_estimacao.animal_UNIQUE'")
[SQL: insert into animal_estimacao (animal) 
                     values ('tartaruga')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro gato. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'gato' for key 'animal_estimacao.animal_UNIQUE'")
[SQL: insert into animal_estimacao (animal) 
                     values ('gato')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi po

A mesma lógica da lista de animais se aplica para as lista de hobbies, climas e bebidas

In [57]:
#Inserir Hobbies
for h in hobbies:
    try:
        #Nessa primeira etapa, vamos inserir na tabela "hobbie" na coluna hobbie, o valor de h.
        querey = f"""insert into hobbie (hobbie) 
                     values ('{h}')"""
        conn.execute(querey)
    except Exception as e:
        print(f"Não foi possível inserir o regristro {h}. O erro foi encontrado foi: {e}")  
print("Hobbies inseridos")


Não foi possível inserir o regristro Praticar esporte. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'Praticar esporte' for key 'hobbie.hobbie_UNIQUE'")
[SQL: insert into hobbie (hobbie) 
                     values ('Praticar esporte')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro Assistir TV. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'Assistir TV' for key 'hobbie.hobbie_UNIQUE'")
[SQL: insert into hobbie (hobbie) 
                     values ('Assistir TV')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro Ler livros. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'Ler livros' for key 'hobbie.hobbie_UNIQUE'")
[SQL: insert into hobbie (hobbie) 
                     values ('Ler livros')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possí

In [58]:
#Inserir Bebidas
for b in bebidas:
    try:
        #Nessa primeira etapa, vamos inserir na tabela "bebida" na coluna bebida, o valor de b.
        querey = f"""insert into bebida (bebida) 
                     values ('{b}')"""
        conn.execute(querey)
    except Exception as e:
        print(f"Não foi possível inserir o regristro {b}. O erro foi encontrado foi: {e}")  
print("Bebidas inseridas")

Não foi possível inserir o regristro Cerveja. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'Cerveja' for key 'bebida.bebida_UNIQUE'")
[SQL: insert into bebida (bebida) 
                     values ('Cerveja')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro Café. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, <NULL>)
[SQL: insert into bebida (bebida) 
                     values ('Café')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro refrigerante. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'refrigerante' for key 'bebida.bebida_UNIQUE'")
[SQL: insert into bebida (bebida) 
                     values ('refrigerante')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro Água. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityErro

In [59]:
#Inserir Clima
for c in climas:
    try:
        #Nessa primeira etapa, vamos inserir na tabela "clima" na coluna clima, o valor de c.
        querey = f"""insert into clima (clima) 
                     values ('{c}')"""
        conn.execute(querey)
    except Exception as e:
        print(f"Não foi possível inserir o regristro {c}. O erro foi encontrado foi: {e}")  
print("Climas inseridas")

Não foi possível inserir o regristro frio. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'frio' for key 'clima.clima_UNIQUE'")
[SQL: insert into clima (clima) 
                     values ('frio')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro quente. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'quente' for key 'clima.clima_UNIQUE'")
[SQL: insert into clima (clima) 
                     values ('quente')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Não foi possível inserir o regristro moderado. O erro foi encontrado foi: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry 'moderado' for key 'clima.clima_UNIQUE'")
[SQL: insert into clima (clima) 
                     values ('moderado')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)
Climas inseridas


Agora as variáveis estão todas inseridas no banco de dados, precisamos agora inserir no banco os dados relacionados da tabela de pesquisa. Para tal, precisamos ler as nossas tabelas que estão guardadas no banco.

Nesse ponto você pode se perguntar: Por qual motivo preciso fazer isso? Elas já não estavam armazenadas aqui no código?

Então, lembra que você criou colunas chamadas de cod_de_alguma coisa?

Para as tabelas de clima, bebida, hobbie e animal nós não inserimos um valor correspondente ao cod_clima, cod_bebida, cod_hobbie e cod_animal.

O que aconteceu então?

Essas colunas foram configuradas no mysql como auto increment e a medida que linhas novas são inseridas elas são incrementadas.

Já para a a tabela de pessoas, já existia um cod_pessoa na nossa tabela de pesquisas.

Leitura das tabelas do banco de dados

In [60]:
df_animais = pd.read_sql('animal_estimacao', con=conn) #ler a tabela animal_estimacao
df_bebidas = pd.read_sql('bebida', con=conn) #ler a tabela bebida
df_climas = pd.read_sql('clima', con=conn) #ler a tabela clima
df_pessoas = pd.read_sql('pessoa', con=conn) #ler a tabela pessoa
df_hobbies = pd.read_sql('hobbie', con=conn) #ler a tabela hobbie

Observe que para os animais, bebidas, climas e hobbies nós inserimos listas mas recebemos data_frames.

Qual o motivo disso?

Lembra das colunas cod_clima, cod_bebida, cod_hobbie e cod_animal? Então, elas foram auto incrementadas dentro do mysql e agora foram lidas.

Observe cada um dos data_frames lidos do banco para fazer uma analogia ao que foi dito.

In [76]:
df_bebidas

,cod_bebida,bebida
0,4,Água
1,2,Café
2,1,Cerveja
3,6,Chá
4,3,refrigerante
5,5,Vinho


In [72]:
df_animais


,cod_animal_estimacao,animal
0,4,cachorro
1,3,gato
2,1,peixe
3,2,tartaruga


In [73]:
df_climas

,cod_clima,clima
0,1,frio
1,3,moderado
2,2,quente


In [74]:
df_pessoas[:5] #mostra as 5 primeiras

,cod_pessoa,genero,data_nascimento
0,10141501,Feminino,1984-05-22
1,10141502,Masculino,1998-06-27
2,10141503,Masculino,1963-02-18
3,10141504,Masculino,1979-09-09
4,10141505,Masculino,1958-10-04


In [75]:
df_hobbies

,cod_hobbie,hobbie
0,4,Aprender algo novo
1,2,Assistir TV
2,7,Dormir
3,5,Escrever
4,8,Escutar música
5,3,Ler livros
6,6,Pintar quadros
7,1,Praticar esporte


In [62]:
#Para facilitar o entendimento criei uma copia do df_pesquisa
#Cópia do Banco
df_pesquisa2 = df_pesquisa.copy()

Agora que já temos as tabelas com seus respectivos códigos, precisamos substituir nas colunas do dataframe df_pesquisa os valores dos códigos das nossas tabelas.

Para isso criamos alguns dicionários para fazer essa tal substituição.

Para facilitar o entendimento, vamos observar a primeira pesquisa:


In [63]:
df_pesquisa2[:1]

,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies
0,10141743,2020-01-01,Masculino,1981-04-25,peixe,frio,Cerveja,Praticar esporte


O que precisamos fazer é o seguinte substituir o valor de:

 peixe pelo seu código 1

 Praticar esporte por seu codigo 1

 frio pelo seu código 1

 cerveja pelo seu código 1

In [64]:
#Usando Dicionários para substituir os valores pelos codigos
#Animais
#Com as funções abaixo é possível criar um dicionário de para os nossos dataframes e substitui-los no df_pesquisa

#Criando dicionário
dict_animais = df_animais.set_index(df_animais.animal)['cod_animal_estimacao'].to_dict()
#Substituindo no dataframe df_pesquisa
df_pesquisa.animal_estimacao = df_pesquisa2.animal_estimacao.map(dict_animais) 
#Observando o que fizemos agora:
df_pesquisa[:5]#mostra as 5 primeiras linhas

#De forma análoga fazemos para os demais dataframes:
#Bebidas
dict_bebidas = df_bebidas.set_index(df_bebidas.bebida)['cod_bebida'].to_dict()
df_pesquisa.bebida_favorita = df_pesquisa2.bebida_favorita.map(dict_bebidas) 

#Hobbies
dict_hobbies = df_hobbies.set_index(df_hobbies.hobbie)['cod_hobbie'].to_dict()
df_pesquisa.hobbies = df_pesquisa2.hobbies.map(dict_hobbies) 

#Clima
dict_climas = df_climas.set_index(df_climas.clima)['cod_clima'].to_dict()
df_pesquisa.clima = df_pesquisa2.clima.map(dict_climas) 

Nosso dataframe df_pesquisa ficou dessa forma agora:

In [65]:
df_pesquisa

,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies
0,10141743,2020-01-01,Masculino,1981-04-25,1,1,1,1
1,10141860,2020-01-01,Feminino,1966-01-19,1,2,2,2
2,10142548,2020-01-01,Masculino,1973-08-02,2,1,2,3
3,10142591,2020-01-01,Masculino,1997-03-05,3,3,1,4
4,10142654,2020-01-01,Feminino,1969-08-05,2,3,2,2
...,...,...,...,...,...,...,...,...
1,10142475,2020-12-31,Masculino,1955-05-30,1,2,2,1
2,10142522,2020-12-31,Masculino,1954-12-22,1,3,1,1
3,10142881,2020-12-31,Masculino,1952-07-30,3,1,3,2
4,10143184,2020-12-31,Masculino,1984-06-05,2,2,3,6


 E observando que os dataframe df_pesquisa possui as colunas cod_pessoa, genero e data_nascimento e que o dataframe df_pessoas possui as mesmas colunas, podemos removê-las e referenciá-las com o cod_pessoa quando fizermos pesquisas no mysql.

In [66]:

#Criei uma variável nova apenas por questões didáticas para comparação entre elas
#Selecionando apenas as colunas de interesse do dataframe df_pesquisa
df_final = df_pesquisa[['cod_pessoa', 'animal_estimacao', 'clima', 'bebida_favorita', 'hobbies', 'data_coleta']]
df_final[:5]#5 primeiras linhas


In [77]:
#Aqui estamos renomeando as colunas do dataframe, novamente com uma nova vairável para questões didáticas de comparação entre elas:
df_final2 = df_final.rename(columns={'animal_estimacao' : 'cod_animal_estimacao', 'clima' : 'cod_clima', 'bebida_favorita' : 'cod_bebida', 'hobbies' : 'cod_hobbie', 'data_coleta' : 'data_pesquisa'})
df_final2[:5]#5 primeiras linhas

,cod_pessoa,cod_animal_estimacao,cod_clima,cod_bebida,cod_hobbie,data_pesquisa
0,10141743,1,1,1,1,2020-01-01
1,10141860,1,2,2,2,2020-01-01
2,10142548,2,1,2,3,2020-01-01
3,10142591,3,3,1,4,2020-01-01
4,10142654,2,3,2,2,2020-01-01


Nas vídeo aulas, o professor usa um script mysql para inserir os dados de pesquisa, exportando o o dataframe df_final2, no entanto eu preri inserir de maneira análoga ao df_pessoas.

In [69]:
for p in df_final2.itertuples():#Um for que vai criar uma variável temporária p do tipo df_final que vai percorrer todo o dataframe df_final
    #observando o dataframe df_final2 podemos notar que, de acordo com as colunas, cada pesquisa possui essas 6 características abaixo
    cod_pessoa = p.cod_pessoa	
    cod_clima = p.cod_clima
    data_pesquisa = p.data_pesquisa
    cod_animal_estimacao = p.cod_animal_estimacao
    cod_bebida = p.cod_bebida
    cod_hobbie = p.cod_hobbie
    #Aqui guardamos as variaveis cod_pessoa, cod_clima, data_pesquisa, cod_animal_estimacao, cod_bebida, cod_hobbie (6 variáveis para 6 colunas)
    try:
        #Nessa primeira etapa, vamos inserir na tabela "pesquisa" nas colunas cod_pessoa, cod_clima, data_pesquisa, cod_animal_estimacao, cod_bebida, cod_hobbie as variaveis que guardamos anteriormente, cod_pessoa, cod_clima, data_pesquisa, cod_animal_estimacao, cod_bebida, cod_hobbie.
        querey = f"""insert into pesquisa (cod_pessoa, cod_clima, data_pesquisa, cod_animal_estimacao, cod_bebida, cod_hobbie) 
                     values ('{cod_pessoa}',  '{cod_clima}', '{data_pesquisa}', '{cod_animal_estimacao}', '{cod_bebida}', '{cod_hobbie}')"""      
        conn.execute(querey)      
        print(f"Registro inserido com sucesso: {cod_pessoa}")
    except Exception as e:        
        print(f"Não foi possível inserir o regristro {cod_pessoa}. O erro foi encontrado foi: {e}")
print("Pesquisas Inseridas")

m sucesso: 10142940
Registro inserido com sucesso: 10143504
Registro inserido com sucesso: 10142071
Registro inserido com sucesso: 10143000
Registro inserido com sucesso: 10143304
Registro inserido com sucesso: 10142180
Registro inserido com sucesso: 10142437
Registro inserido com sucesso: 10142937
Registro inserido com sucesso: 10142959
Registro inserido com sucesso: 10142996
Registro inserido com sucesso: 10143127
Registro inserido com sucesso: 10143135
Registro inserido com sucesso: 10143216
Registro inserido com sucesso: 10143470
Registro inserido com sucesso: 10141568
Registro inserido com sucesso: 10141686
Registro inserido com sucesso: 10141791
Registro inserido com sucesso: 10142074
Registro inserido com sucesso: 10142665
Registro inserido com sucesso: 10143049
Registro inserido com sucesso: 10143178
Registro inserido com sucesso: 10142044
Registro inserido com sucesso: 10142311
Registro inserido com sucesso: 10142533
Registro inserido com sucesso: 10142642
Registro inserido co